# Regression

## *Workshop 7*  [![Open In Colab](https://github.com/oballinger/QM2/blob/main/colab-badge.png?raw=1)](https://colab.research.google.com/github/huanfachen/DSSS/blob/main/Week_7/Practical_07_linear_regression.ipynb)

### Aims:

In this workshop, we're going to be modeling the relationship between education and income. More precisely, we're going to be looking at the effect of increasing education on hourly wages using Ordinary Least Squares regression. We're going to accomplish this in four steps: 

1. Summary Statistics
    * Table of Summary Statistics
2. Visualisation 
    * Exploratory Plots
3. Assumptions
    * A. Independence
    * B. Heteroscedasticity: Regression plots + Q-Q plot
    * C. Multicollinearity: VIF + Correlation Matrix 
4. Regression
    * Regression Table

If you're conducting a regression, you must complete the steps above, and produce each item indicated by a bullet point. 

## Getting Started


As always we'll start by importing the libraries I need

In [ ]:
#This tells python to draw the graphs "inline" - in the notebook
%matplotlib inline  
import matplotlib.pyplot as plt
import statsmodels.api as sm
from math import sqrt
from numpy.random import seed
from numpy.random import randn
from numpy import mean
from scipy.stats import sem
import statistics 
import seaborn as sns
from IPython.display import display, Math, Latex, display_latex
import plotly.express as px
import pylab
import pandas as pd
import numpy as np
# make the plots (graphs) a little wider by default
pylab.rcParams['figure.figsize'] = (10., 8.)
sns.set(font_scale=1.5)
sns.set_style("white")



Now that I've imported the libraries I'm going to be using, I'm ready to import the data: 

In [ ]:
df=pd.read_csv('https://raw.githubusercontent.com/huanfachen/DSSS/main/data/cps.csv')
df.head()

Our dataframe has 10 columns:

1. *year*: Survey year
2. *age*: the person's age
3. *sex*: the person's sex 
    * 1=male
    * 2=female
4. *race*: the person's race 
    * White non hispanic=1
    * Black non hispanic=2
    * Hispanic=3
    * Other non hispanic=4)
5. *sch*: Educational attainment
    * None = 0, 
    * Grades 1-12 = 1-12
    * Some University = 13, 
    * Associate's degree = 14, 
    * BA = 16
    * Advanced Degree = 18
6. *union*: Union membership 
    * N/A = 0, 
    * No union coverage = 1, 
    * Member of labor union=2, 
    * Covered by union but not a member=3
7. *incwage*: Wage and salary income
8. *realhrwage*: Real Hourly Wage
9. *occupation*: Occupation
10. *ind*: [industry code](https://www.census.gov/naics/?58967?yearbck=2002)
11. *state*: [FIPS code](https://www.bls.gov/respondents/mwr/electronic-data-interchange/appendix-d-usps-state-abbreviations-and-fips-codes.htm) denoting the state of residence.

We'll begin, as we did with last week's workshop, by selecting the year 2013 in our data and making sure that all the variables that represent categories are stored as categorical in python:

In [ ]:
reg_df=df[df['year']==2013].drop(['year'],axis=1) # filter the whole dataset to 2013 and drop year column
reg_df[['race','union','sex','occupation','ind','state']]=reg_df[['race','union','sex','occupation', 'ind','state']].astype('category') # convert these columns to categorical

## 1. Summary Statistics

Once our data has been cleaned and all our variables are stored as the appropriate type, we can start with the first step of any regression project: creating a table of summary statistics. This is an important part of the process, since it gives the reader a qualitative understanding of your data before you analyze it. It also serves to demonstrate that you've cleaned the data appropriately, and that the measures of the variables make sense.

In [ ]:
summary=reg_df.describe().round(2)  # generate summary statistics, and round everything to 2 decimal degrees
summary=summary.T #.T transposes the table (rows become columns and vice versa)
summary

This table is already informative. I now know that the average person in this dataset is 42 years old, has around 14 years of schooling, and makes $24/hour (or $51,821/year). However, it's also useful to spot potential errors in data entry that may warrant greater attention. 

Notice the max value for real hourly wage. Despite the fact that those in the top 75% of earners make $28.12/hour, someone is making $34,760 per hour. Must be nice (or, may be a data entry error). Either way, because regresisons are sensitive to this sort of outlier, we should remove it. I've defined a function below that calculates the quartiles and filters out observations that are more than three times as far away form the top quartile as the top quartile is from the bottom one. This was a somewhat arbitrary choice, but it allows me to be consistent if I want to apply it to other variables. You could also just pick a cutoff qualitatively and justify it (e.g. "I will focus on those making up to $250k per year, since they represent the population i'm trying to understand").   

In [ ]:
def filter_outliers(var):
    q1 = var.quantile(0.25) # calculate the first quartile
    q3 = var.quantile(0.75) # calculate the third quartile
    iqr = q3 - q1 # calculate the interquartile range
    low = q1 - 3*iqr # calculate the lower bound
    high = q3 + 3*iqr # calculate the upper bound
    filtered = reg_df[(var > low) & (var < high)] # filter  the values that are within the bounds
    dropped_observations= len(var)-len(filtered) # calculate the number of observations that were dropped

    print('Dropped {} observations'.format(dropped_observations))
    return  filtered

reg_df=filter_outliers(reg_df['realhrwage']) # filter outliers from realhrwage


We can see that this operation dropped 1040 observations that had extreme values in the "realhrwage" variable. Let's re-generate the table of summary statistics and only keep four columns: count, mean, standard deviaiton, minimum, and maximum.

In [ ]:
summary=reg_df.describe().round(2).T
summary[['count','mean','std','min','max']]

## 2. Visualization 

The summary statistics table provides us with a good overview of some of the variables we're interested in. However, you'll notice that it omits many of the other variables in our dataset: the categorical ones. This is because calculating the mean, standard deviation, etc. of something like the "occupation" column doesn't really make sense. For that, we turn to visualization. 

### Visualizing the distribution of categorical variables 

So far in this course we've been using a python library called Matplotlib to make our visualizations, which we've been calling using the 'plt' alias. But this isn't the only one that is avaialble to us. [Seaborn](https://seaborn.pydata.org/) is another library that has some cool plotting functions that are more geared towards statistical analysis. We've already imported seaborn above, and we'll be calling it using the alias "sns". We can use it in conjunction with matplotlib. 

To get a sense of the distribution of our categorical variables, we'll make some plots that count the number of observations in each category. Let's start with the race category:

In [ ]:
sns.countplot(data=reg_df, x='race') # plot the union variable

plt.title('Race') # add a title
plt.xlabel('') # remove the x axis label
plt.xticks(ticks=[0,1,2,3],labels=['White', 'Black','Hispanic','Other']) # replace the x axis labels with more descriptive labels
plt.show() # show the plot


### Exercise

1. Generate an equivalent plot for the other categorical columns 
2. What is the most common industry code, and what does it correpsond to? 


## 3. Assumptions 

Once you've generated summary statistics for your continuous variables and exploratory plots for the categorical ones, it's time to start thinking about the relationships *between* the variables. Today, we're going to be modeling a linear relationship between income and years of schooling, by means of a **linear regression**. But before we do that, we need to check a couple things-- all statistical tests have a number of assumptions that must be satisfied in order to yield robust results. Before we run a regression, we must check that the assumptions in this case are satisfied. There are four main ones:

    A. Indepdendence 
    B. Homoscedasticity
    C. Multicollinearity 

Let's go through them one by one. 

### A. Independence 

**`Linear regression assumes that measurements for each sample subject are in no way influenced by or related to the measurements of other subjects.`**

Though in the full CPS dataset we have repeat observations of the same individual over time, we've only been analyzing one year's worth of data, so we satisfy the independence assumption. If we ran a regression on the full sample over multiple years, *this would violate the independence assumption*. It's very possible to run a regression with repeat observations of the same units (people, places, etc.) over time, but you need to use a special type of regression called a **panel regression**. More on that next week. 

### B. Homoscedasticity 

**`Linear regression assumes that the variance of residuals is the same for any value of $x$, and that residuals are normally distributed with a mean of 0.`**

This is a complicated way of saying your regression line should fit consistently across the full range of $x$ values. If there are really small residuals (i.e., all the data points are close to the line) for low values of $x$, but larger residuals for high values of $x$, the regression is not performing well-- we wouldn't have the same confidence in our predictions at different values of $x$. Similarly, if all the residuals are on one side of the regression line in different parts of the $x$ range, the model will consistently over/underestimate in those regions. When the variance of residuals from a regression model are inconsistent, we have **`Heteroscedasticity`**. 

We can explore potential heteroscedasticity by visually inspecting a regression plot. In our case, we're primarily interested in the relationship between years of schooling and hourly wages, so we'll be plotting these variables against eachother. `sns.jointplot()` lets us create a plot with four components which can help us diagnose potential heteroscedasticity:

- The main plot is a scatterplot between hourly wages on the y axis, and years of schooling on the x axis. 
- A regression line overlaid on this plot lets us see the relationship between our model and the underlying data 
- A histogram to the right of the plot shows the distribution of the hourly wages variable, which is heavily skewed.
- A histogram above the plot shows the distribution of the years of schooling variable, which has an almost bimodal form.   

In [ ]:
sns.jointplot(data=reg_df, # plot a scatterplot with a regression line and two histograms
                x='sch', # set the x axis to be the years of schooling
                y='realhrwage', # set the y axis to be the hourly wage
                kind="reg",  # set the kind of plot to be a regression plot
                scatter_kws=dict(alpha=0.1), # set the transparency of the points to be 0.1 (10%)
                line_kws=dict(color='red'), # set the color of the regression line to red
                height=10) # set the height of the plot to be 10 inches 

plt.xlabel('Years of Schooling') # add a label to the x axis
plt.ylabel('Hourly Wage') # add a label to the y axis

The plot above is cause for concern. From 0 to 5 years of schooling the model has underestimated hourly wages for every single observation. Conversely, at the far right tip of the regression line, we can see that the model *overestimates* income for many individuals with 18 years of schooling. This gives us reason to suspect that there may be asymmetry in the residuals of our model (heteroscedasticity). We're going to fix this in the Exension section below. But for now, let's proceed. 

### C. Multicollinearity 

**`Multicollinearity emerges when two or more independent variables which are highly correlated are included in a model.`** A key goal of regression analysis is to isolate the relationship between each independent variable and the dependent variable. The interpretation of a regression coefficient is that it represents the mean change in the dependent variable for each 1 unit change in an independent variable when you hold all of the other independent variables constant. 

The idea is that you can change the value of one independent variable and not the others. However, when independent variables are correlated, it indicates that changes in one variable are associated with shifts in another variable. The stronger the correlation, the more difficult it is to change one variable without changing another. See this [blog post](https://statisticsbyjim.com/regression/multicollinearity-in-regression-analysis/) for a thorough explanation.

One way of visually exporing multicollinearity is through a correlation matrix:

In [ ]:
sns.heatmap(reg_df.select_dtypes(include=np.number).corr(), # plot a correlation matrix 
            annot=True, # show the correlation values on the plot
            fmt=".2f", # set the format of the correlation values to be two decimal places
            cmap='coolwarm') # set the color palette to be coolwarm (blue for negative correlations, red for positive correlations)

plt.title('Correlation Matrix') # add a title

This matrix has each of the continuous variables in `reg_df` on both axes. Each cell denotes the correlation between the corresponding variables. Naturally, on the diagonal we have a series of perfect correlations (1.00), as each variable is perfectly correlated with itself. `incwage` (annual salary) and `realhrwage` (hourly wage) are highly correlated with each other, which makes a lot of sense. This isn't a concern for multicollinearity, though, since `realhrwage` will be our dependent variable. This type of correlation matrix is also a good way of conducting exploratory data analysis-- we can already see that the next-highest set of correlations is between years of schooling and both hourly wages and annual salary. 

Though a very high correlagtion coefficient between independent variables is a cause for concern, the formal way of dealing with muticollinearity is through the use of the **`Variance Inflation Factor (VIF)`**. VIF is the ratio of the variance in a model with multiple predictors by the variance of a model with a single predictor: 

$$\large VIF_j=\frac{1}{1-R_{j}^{2}}$$

VIFs start at 1 and have no upper limit. A value of 1 indicates that there is no correlation between this independent variable and any others. VIFs between 1 and 5 suggest that there is a moderate correlation, but it is not severe enough to warrant corrective measures. VIFs greater than 5 represent critical levels of multicollinearity where the coefficients are poorly estimated, and the p-values are questionable. More explanation of the theory can be found [here](https://en.wikipedia.org/wiki/Variance_inflation_factor). 

Below is a function that calculates VIF for each independent variable in a dataframe, and drops them if they exceed a threshold (set to 5).

In [ ]:
# calculating VIF
# This function is amended from: https://stackoverflow.com/a/51329496/4667568

from statsmodels.stats.outliers_influence import variance_inflation_factor 
from statsmodels.tools.tools import add_constant

def drop_column_using_vif_(df, list_var_not_to_remove=None, thresh=5):
    '''
    Calculates VIF each feature in a pandas dataframe, and repeatedly drop the columns with the highest VIF
    A constant must be added to variance_inflation_factor or the results will be incorrect

    :param df: the pandas dataframe containing only the predictor features, not the response variable
    :param list_var_not_to_remove: the list of variables that should not be removed even though it has a high VIF. For example, dummy (or indicator) variables represent a categorical variable with three or more categories.
    :param thresh: the max VIF value before the feature is removed from the dataframe
    :return: dataframe with multicollinear features removed
    '''
    while True:
        # adding a constatnt item to the data
        df_with_const = add_constant(df)

        vif_df = pd.Series([variance_inflation_factor(df_with_const.values, i) 
               for i in range(df_with_const.shape[1])], name= "VIF",
              index=df_with_const.columns).to_frame()

        # drop the const as const should not be removed
        vif_df = vif_df.drop('const')
        
        # drop the variables that should not be removed
        if list_var_not_to_remove is not None:
            vif_df = vif_df.drop(list_var_not_to_remove)
            
        print('Max VIF:', vif_df.VIF.max())
        
        # if the largest VIF is above the thresh, remove a variable with the largest VIF
        if vif_df.VIF.max() > thresh:
            # If there are multiple variables with the maximum VIF, choose the first one
            index_to_drop = vif_df.index[vif_df.VIF == vif_df.VIF.max()].tolist()[0]
            print('Dropping: {}'.format(index_to_drop))
            df = df.drop(columns = index_to_drop)
        else:
            # No VIF is above threshold. Exit the loop
            break

    return df

Now we can implement this on our dataset:

In [ ]:
ind_vars=['sex','age','sch', 'union','race']

vif = drop_column_using_vif_(reg_df[ind_vars], thresh=5)
print("The columns remaining after VIF selection are:")
print(vif.columns)

The maximum VIF value encountered was 1.04-- well within the acceptable range. Accordingly, the function hasn't dropped any of the independent variables in our dataset. 

Having explored our data through visualizations and summary statistics, and checked the assumptions of linear regression, we're now ready to begin building a model. 

## 4. Regression

Remember, the Ordinary Least Squares (OLS) regression seeks to find a straight line that best describes the relationship between two variables: 

$$y= \beta_0 + \beta_1x+\epsilon $$

In our case, we're trying to predict hourly income-- this is our **dependent variable**, and there can be only one per regression. The variable we're using to predict hourly income is years of schooling, which is our **independent variable**. We can have multiple of these per regression. As such, the regression equation in our scenario looks like this: 

$$Hourly\ Income= \beta_0 + \beta_1 \times Years\ of\ Schooling +\epsilon $$

Because the regression model will estimate the parameters $\beta_0, \beta_1$ and $\epsilon$, we just need to supply python with $x$ and $y$; We can do so by passing `realhrwage ~  sch` to the `ols()` function from statsmodels. This will run a regression of the form specified above, which we will store in an variable called `model`. We can get the output from this model using `model.summary()`:

In [ ]:
from statsmodels.formula.api import ols
from statsmodels.iolib.summary2 import summary_col

model= ols('realhrwage ~  sch', data=reg_df).fit() # fit the model
print(model.summary()) # print the summary


There's a lot going on in the regression output above. If you want a more detailed explanation of what each part means, check out this [blog post](https://medium.com/swlh/interpreting-linear-regression-through-statsmodels-summary-4796d359035a). In practice, we only need to focus on a couple parts of this output:

* `R-squared`: This value tells the proportion of the variation in our dependent variable (realhrwage) that is explained by the model we fit. In this case we can interpret it as follows: 
    * **18.1% of the variation in hourly wages can be explained by this regresion model**

* `coef`: These are our $\beta$ estimates; it is the slope of the regression line that describes the relationship between a given independent variable (sch) and the dependent variable (realhrwage). There are two coefficients listed under this 
    * `sch`: This is $\beta_1$, the slope coefficient on the years of schooling variable. It tells us the change in $y$ that results from a 1-unit increase in $x$. In robotic terms, we can interpret it as follows: 
        * **A 1 unit increase in `sch` leads to a 2.0327 increase in `realhrwage`**. But we are not robots, and both of these variables are in units that we can interpret in plain english. Here's a more natural interpretation:
        * **On average, every additional year of schooling is associated with a $2.03 increase in hourly wages.** 

    * `Intercept`: This is $\beta_0$. It tells us the value of $y$ when all of the independent variables in the model are held at 0. In this case, it can be interpreted as
        * **According to our model, a person with 0 years of schooling is predicted to earn -$6.62 per hour**
        * Naturally, this is a nonsensical prediction. There are no jobs that pay negative wages. We'll examine why this is happening in the next section, when we look into the assumptions of linear regression. 
    
    
* `P>|t|`: this is known as the "p-value", and is the main measure of statistical significance. **A p-value denotes the probability of obtaining a result at least as extreme as the one observed, assuming that the null hypothesis is true**. In the case of a regression, the null hypothesis is that there is no relationship between our variables-- increasing $x$ has no effect on $y$. In other words, that the regression line is flat: $\beta_1=0$ . A p-value of 0.05 means that the coefficient is statistically significant at the 5% level. In our case, the p-value is 0.000 (note: this doesn't mean it's equal to zero, just very very small), and we can therefore reject the null hypothesis that $\beta_1=0$ at the 1% confidence level. However, this isn't the end of the story-- remember our weird negative intercept, and the fact that our model explains less than 20% of the variation in hourly wages ($R^2=0.181$). For a good overview of what exactly a p-value is, and why we should be cautious when interpreting them, see this [journal article](https://www.ncbi.nlm.nih.gov/pmc/articles/PMC6532382/).


### Categorical Variables

The results of our first regression seem to show that the more education a person has, the higher their hourly wage. This makes intuitive sense, but it's probably not the whole picture. We may also suspect that older people earn more, since they have more experience and are more senior. We've also seen i previous classes that there are significant disparities in income. Considering we have data on all these variables, we can set up the following model:

$$Hourly\ Income= \beta_0 + \beta_1 \times Years\ of\ Schooling + \beta_2 \times Age + \beta_3 \times Sex +\epsilon $$

When we convert this equation into the python equivalent, it will look like this: 

`realhrwage ~  sch + age + C(sex)`

Notice that for the sex variable is put within `C()`. This is how we indicate that the variable in question is categorical, and that it should be treated differently. Unlike a continuous variable, we're not interested in the change in $y$ that results from a 1 unit increase in $x$, since our units have no meaningful order. Instead, we'll have to pick one of the categories (called a **base category**/**reference category**), and compare each of the other categories in that variable against this one. You can specify the base category explicitly, or python will pick one for you. As such, for a categorical variable with $n$ categories, we get $n-1$ coefficeints which denote the change in $y$ associated with membership of a given category compared to the base category. For example, if we have a categorical variable with three levels $a, b, c$ where $a$ is the base category, we would get *two* coefficients: $\beta_1 b$ and $\beta_2 c$. Then we would interpret the resulting coefficient as 

* "Compared to category $a$, membership of category $b$ is associated with a $\beta_1$ change in $y$."
* "Compared to category $a$, membership of category $c$ is associated with a $\beta_2$ change in $y$."

Let's see what this looks like in our regression output:

In [ ]:
model = ols('realhrwage ~  sch + age + C(sex)', data=reg_df).fit() 
print(model.summary())

We now have 4 coefficients. In general, we don't always have to interpret the Intercept coefficient. It's not really that meaningful in this case, since now it denotes the predicted hourly income of someone who is male, has 0 years of schooling, and is 0 years old. It's good to keep it in mind as a sense check, though. The rest of the coefficients can be interpreted as follows: 

* `C(sex)[T.2]`: On average, women earn $5.2 less per hour than men. 
    * [T.2] in this line denotes the category in this variable associated with the given coefficient. So this is telling us that what is being shown is the coefficient associated with membership of category 2 in the sex variable; based on the description of the variables above, we know that sex=1 indicates men, and sex=2 indicates women. Naturally, we don't see a coefficient for `C(sex)[T.1]`, because this is the *base category*. 
* `sch`: Every additional year of schooling is associated with a $2.13 increase in hourly income
* `age`: Every additional year of age is associated with a $0.16 increase in hourly income

### Exercise

1. Estimate a regression of the following form and store the results in a variable called **model1**:

$$Hourly\ Income= \beta_0 + \beta_1 \times Years\ of\ Schooling + \beta_2 \times Age + \beta_3 \times Sex + \beta_4 \times Union\ Membership + \beta_5 \times Race +\epsilon $$

2. Intepret each of the coefficients appropriately. Make note of the statistical significance of each result, and comment on the overall fit of the model.


### Creating a Regression Table 

Now that we've got a good sense of how regressions work and how to interpret them, we need to communicate these results properly. Many of you have probably read journal articles in which regression results are reported, but I doubt you've ever seen the output of `model.summary()` copied and pasted in the text of an article. Instead, these results are reported following a fairly standardized convention: a regression table. It picks out the components of the model summary that we're interested in, and formats them in a consistent and easy-to-interpret way. Luckly, the statsmodels package has a function called `summary_col` that takes a fitted model and formats it for us automatically; we just need to tweak a few options. 

In the example below, i'm going to run two regressions; one in which i filter the data to only include people from California, and another for people in Mississippi (the richest and poorest states, respectively), to see if the relationship between wages, sex, age, and schooling differ geographically. I'm then going to create a regression table in which each column is a different regression model, and row will contain the coefficient for a given independent variable with the standard error in parentheses underneath and the level of statistical significance (i.e., size of the p-value) denotes by stars such that: * p<0.05, ** p<0.01, *** p<0.001.

In [ ]:
california = ols('realhrwage ~  sch + age + C(sex)', data=reg_df[reg_df['state']==6]).fit()  # fit a model to california-- i'm filtering the data using the FIPS code for california, which is 6
mississippi = ols('realhrwage ~  sch + age + C(sex)', data=reg_df[reg_df['state']==28]).fit()  # same thing for mississippi (FIPS code 28)

table=summary_col( # create a regression table 
    [california,mississippi], # pass the models to the summary_col function
    stars=True, # add stars denoting the p-values of the coefficient to the table; * p<0.05, ** p<0.01, *** p<0.001
    float_format='%0.3f', # set the decimal places to 3
    model_names=['California','Mississippi'], # set the name of the model
    info_dict = {"N":lambda x: "{0:d}".format(int(x.nobs))}) # add the number of observations to the table

print(table)

This layout lets us clearly explore our regresison results. This lets us clearly compare the coefficients of the same variable in different models. For example, we can see that men tend to earn $5.15 more per hour than women in California, but just $4.98 more per hour in Mississippi, and both of these results are statistically significant at the 1% level. This suggests that the wage gap is actually somewhat higher in California! Why might this be?

### Exercise 

$$ Hourly\ Income= \beta_0 + \beta_1 \times Years\ of\ Schooling + \beta_2 \times Age + \beta_3 \times Sex + \beta_4 \times Union\ Membership + \beta_5 \times Race +\epsilon $$

1. Run five regressions, each of the form above (same as earlier):
    * In the first model, run the regression on the full sample contained in `reg_df`. In subsequent modles, restrict the sample to the following professions:
        * Production
        * Farmers
        * Bankers
        * Doctors & Lawyers
2. Create a regression table containing the results of each model in a separate column 
3. Interpret the coefficients on the union related variables 
    * How does union membership affect hourly wages across different sectors?
    * How does the gender wage gap vary across sectors?

## Extension

Though we've gotten some significant results and interesting insights from our modeling effort so far, we can further improve our model. In particular, we may want to revisit the way we've defined some of our variables, since we suspect that we may have some heteroscedasticity in our models, and have consequently been getting some weird results (e.g. negative hourly income). Let's have a closer look at our main independent variable (years of schooling), and 

### Years of Schooling

Let's start with the main independent variable, Years of Schooling. Is it appropriate to think of this as a continuous variable, linearly related to hourly wages? A linear relationship should be consistent across the full range of data-- in other words, the increase from 0 years of schooling to 1 year of schooling should have the same effect on wages as the increase from 11 to 12 years of schooling, even though this one year determines whether a person has a highschool diploma or not. We would probably expect the increase from 11 to 12 years of schooling to have a greater effect on wages than the change from 0 to 1 years of schooling. 

We could conceptualize of years of schooling instead as a categorical or ordinal variable. Below, i've used the `np.where()` function to create a new variable called `degrees` based on years of schooling such that all observations have 0 

In [ ]:
reg_df['degrees']=np.where(reg_df['sch']==12, 1,0) # create a new variable called degrees that is equal to 1 if the person has 12 or more years of schooling and 0 otherwise
reg_df['degrees']=np.where(reg_df['sch']==16, 2, reg_df['degrees']) # if the person has 16 years of schooling, set the degrees variable to 2

#### Exercise

1. Add a level to the `degrees` column that denotes having a PhD.
2. Run a regression in which hourly wages is the dependent variable and degrees is a **continuous** independent variable, and interpret. 
3. Run the same regression again, but treat `degrees` as categorical. How does the interpretation change?


### Hourly Wages

When checking the regression assumptions, we suspected that there may be some heteroscedasticity-- i.e., that our model performs better in some regions of the $x$ distribution compared to others; remember, it consistently underestimated hourly income for those with little/no schooling, as evidenced by the negative intercept and the regression scatterplot:

In [ ]:
sns.jointplot(data=reg_df, x='sch', y='realhrwage', kind="reg",  scatter_kws=dict(alpha=0.1), line_kws=dict(color='red'), height=10)



We can more thoroughly diagnose heteroscedasticity *after* having run our regression models, since we have access to the model's **residuals** (the difference between the observed values and the predicted values). Remember, one of the assumptions of linear regression is that the residuals are normally distributed. A Quantile-Quantile Plot (Q-Q Plot) is a plot of the quantiles of a sample against the quantiles of a theoretical distribution. The quantiles are the values that divide the range of a probability distribution into continuous intervals with equal probabilities. Thus, we can use a Q-Q plot to compare the residuals of our model to a normal distribution as follows: 

In [ ]:
model = ols('realhrwage ~  sch', data=reg_df).fit()  # fit a model
residuals = model.resid # get the residuals

# make the figure wider
plt.rcParams["figure.figsize"] = [20, 10]

f, axes = plt.subplots(1, 2)
sns.histplot(residuals, kde=True, ax=axes[0]) # plot the residuals
axes[0].set_title('Histogram of Residuals') # add a title

sm.qqplot(residuals, line='45', fit=True,  ax=axes[1]) # plot the residuals
axes[1].set_title('Q-Q Plot') # add a title

plt.show() # show the plot

This Q-Q plot suggests that our residuals are not normally distributed, as very few of them are on the red line. This is probably due to the fact that the `realhrwage` variable is itself highly skewed. 

Log transformations are often recommended for skewed data, such as monetary measures or certain biological and demographic measures. Log transforming data usually has the effect of spreading out clumps of data and bringing together spread-out data. So instead of:

$$Hourly\ Income= \beta_0 + \beta_1 \times Years\ of\ Schooling +\epsilon $$

we get: 

$$\log{(Hourly\ Income)}= \beta_0 + \beta_1 \times Years\ of\ Schooling +\epsilon $$

In effect, this means changing our belief that there is a linear relationship between schooling and income (a constant increase in x leads to a constant increase in y across the whole range of x). Qualitatively, this means 

In [ ]:
reg_df['logwage']=np.log(reg_df['realhrwage'])
sns.jointplot(data=reg_df, x='sch', y='logwage', kind="reg",  scatter_kws=dict(alpha=0.1), line_kws=dict(color='red'), height=10)

A few things are noticeably different in this plot. First, the histogram of `logwage` on the far right is a lot less skewed than the histogram of `realhrwage`. Consequently, the regression line seems to fit the data slightly better across the whole range of the data. 

We can generate the same residual histogram and Q-Q plot as before, but using a model in which `logwage` is the dependent variable:

In [ ]:
log_model = ols('logwage ~  sch', data=reg_df).fit()  # fit a model
log_model_residuals = log_model.resid # get the residuals

# make the figure wider
plt.rcParams["figure.figsize"] = [20, 10]

f, axes = plt.subplots(1, 2)
sns.histplot(log_model_residuals, kde=True, ax=axes[0]) # plot the residuals
axes[0].set_title('Histogram of Residuals') # add a title

sm.qqplot(log_model_residuals, line='45', fit=True,  ax=axes[1]) # plot the residuals
axes[1].set_title('Q-Q Plot') # add a title

plt.show() # show the plot

It's not perfect, but it's a lot better than the unlogged version; a large proportion of the residuals fall on the red line in the Q-Q plot, though they diverge at the tips. The histogram of residuals also seems to be less skewed, and more evenly distributed around 0. 

## Coefficient interpretation. 


Only the dependent/response variable is log-transformed. Exponentiate the coefficient, subtract one from this number, and multiply by 100. This gives the percent increase (or decrease) in the response for every one-unit increase in the independent variable. Here's a [full guide](https://data.library.virginia.edu/interpreting-log-transformations-in-a-linear-model/#:~:text=Interpret%20the%20coefficient%20as%20the,variable%20increases%20by%20about%200.20%25.) to interpreting the coefficients on log-transformed variables. 

First, let's compare the unlogged and logged models:

In [ ]:
table=summary_col( # create a regression table 
    [model,log_model], # pass the models to the summary_col function
    stars=True, # add stars denoting the p-values of the coefficient to the table; * p<0.05, ** p<0.01, *** p<0.001
    float_format='%0.3f', # set the decimal places to 3
    model_names=['Unlogged','Logged'], # set the name of the model
    info_dict = {"N":lambda x: "{0:d}".format(int(x.nobs))}) # add the number of observations to the table

print(table)

Interestingly, we can see that we've also got a 1% increase in $R^2$ just from logging the dependent variable. While the coefficient for schooling can be interpreted normally for the unlogged model (every additional year of schooling leads to a $2.03 increase in hourly wages), this is not the case for the logged model. We can interpret the coefficeint in the logged model as follows: 

In [ ]:
b1=log_model.params.sch # get the coefficient for sch
exp_b1=np.exp(b1) # exponentiate the coefficient

pct_change=(exp_b1-1)*100 # multiply by 100 to get the percentage change
print('For every additional year of schooling, log wages increase by {}%'.format(round(pct_change,2)))

----------------------
## Panel Regression 

[Surveys](https://www.pewresearch.org/politics/2012/04/17/with-voters-focused-on-economy-obama-lead-narrows/) indicate that "jobs" are consistently one of the most important issues among voters in U.S. presidential elections, and that Republicans are [typically perceived](https://thehill.com/homenews/campaign/3700047-republicans-hold-14-point-advantage-on-which-party-would-do-better-job-on-economy-poll/) as better in handling the economy than Democrats. An [article](https://www.nbcnews.com/news/us-news/where-did-trump-make-election-gains-unemployment-data-tells-surprising-n1247935) in NBC claims that "analysis of unemployment and voting data found that the president’s share of the vote held steady or increased in each of the 20 counties with the highest rise in unemployment from September 2019 to September 2020. And his vote share improved by 1 percentage point or more in 70 of the 100 hardest-hit counties." Let's look into this. 


### Data Collection

There are only 50 states in the U.S. but there are over 3000 counties-- this allows us to increase our sample size and perform a more fine-grained analysis. This is particularly important if we're interested in investigating the relationship between unemployment and voting behaviour, because of the urban-rural divide. For example, with in the state of New York there are probably vast differences in social and economic factors relevant to voting behaviour between Manhattan and very rural areas; this variation is lost when we look at aggregate state-level resutls, but visible when we look at the county-level. As such, in addition to the datasets we've just imported, we're going to be downloading county-level unemployment data straight from the Github repo. 

In [ ]:
counties=pd.DataFrame() # create empty dataframe to store the county data for each year

for year in range (1990,2022): # loop through years from 1990 to 2021
    annual=pd.read_csv('https://raw.githubusercontent.com/huanfachen/DSSS/main/data/labour_force_county/laucnty'+str(year)[-2:]+'.txt', sep="\s{2,}", header=3, on_bad_lines='skip', engine='python', skipfooter=2) # read in the data for each year by adding the last two digits of the year to the url
    annual=annual[['Code','Code.1','Year','Rate','Force']].rename(columns={'Code':'state','Code.1':'county','Year':'year','Rate':'unemployment','Force':'population'}) # rename the columns
    counties=pd.concat([counties,annual]) # append the data for each year to the dataframe

counties['county_fips']=counties['state'].astype(str).str.zfill(2)+counties['county'].astype(int).astype(str).str.zfill(3) # create a fips code for each county
counties['unemployment']=pd.to_numeric(counties['unemployment'], errors='coerce') # convert the unemployment rate to a numeric variable
counties['population']=pd.to_numeric(counties['population'].str.replace(',',''), errors='coerce')/1000 # convert the population to a numeric variable and divide by 1000 to get the population in thousands
counties.dropna(inplace=True) # drop missing values
counties.head()

Part of the cleaning process in the cell above involves the creation of a column called "county_fips"-- this stands for [Federal Information Processing System](https://transition.fcc.gov/oet/info/maps/census/fips/fips.txt#:~:text=FIPS%20codes%20are%20numbers%20which,to%20which%20the%20county%20belongs.). This is a code that uniquely identifies states and counties in the U.S. A two digit FIPS code identifies states (e.g. 01: Alabama, 02: Alaska, etc.) and a five digit fips code identifies counties (e.g. 010001: Atauga County, Alabama; 02068: Denali Borough, Alaska). Notice, the first two digits of the five-digit county FIPS code indicates the state. Boring, yes, but these codes are imperative in allowing us to join county- and state- level datasets from different sources quicky and easily. Imagine what a nightmare it would be to try to join them using the names of the counties, having to deal with capitalizations, punctuation, etc. Yikes. 

### Maps 

Great-- we've now got clean, county-level unemployment and population data spanning from 1990-2022 on an annual basis. Lets make a map to explore the spatial distribution of unemployment across time in the U.S. In order to do that, we're going to need a spatial file that tells us the shapes of the counties; I've imported it as a variable called `county_polygons`. We're then going to create an  map using the [Plotly](https://plotly.com/python/) library, which is great for making pretty, interactive maps and plots. It will have a slider on the bottom that lets us view unemployment in different years. It's doing quite a bit under the hood so it will take some time to plot. Be patient. 

In [ ]:
import json
!curl https://raw.githubusercontent.com/huanfachen/DSSS/main/data/US_county/geojson-counties-fips.json ./geojson-counties-fips.json
county_polygons = json.load(open('geojson-counties-fips.json'))

In [ ]:
plot_sample=counties[counties['year']>2007] # subset the data to only include years after 2007 -- it would take too long to plot all of the data

px.choropleth( # plot a choropleth map using the plotly express (px) library
                plot_sample, # load the dataframe
                locations='county_fips', # set the location column to the state code
                geojson=county_polygons, # set the location mode to USA states (you could add your own custom geojson/spatial file here)
                scope='usa', # set the scope to the USA, so that it only plots the states
                color="unemployment", # set the color of the states to correspond to the unemployment rate
                animation_frame=plot_sample["year"].astype(str), # set the animation frame to the date, creating a slider at the bottom of the map
                color_continuous_scale=px.colors.sequential.Viridis, # set the color scale to Viridis, a commonly used color scale
                range_color=[0, 10], # set the range of the color scale to 0-10
                height=1000) # set the height of the map to 1000 pixels

This map is interactive-- meaning you can zoom in, pan around, and hover over it to get further information on the unemployment level in each county. You can also use the slider at the bottom to toggle between different years; if you move the slider from 2008 to 2009, you'll see lots of yellow suddenly appearing. A similar thing happens between 2019 and 2020. What's going on? Play around with this map for a second, and make note of spatial and temporal trends in unemployment. 

Now we're going to do the same thing for the elections data, which I've taken the liberty of cleaning. Let's load it up as a dataframe called `elections`, and make another map in which we plot vote shares in various elections such that red shows republican support, and blue shows democratic support. 

In [ ]:
elections=pd.read_csv('https://raw.githubusercontent.com/huanfachen/DSSS/main/data/US_county/elections.csv',converters={'county_fips': str})
px.choropleth( 
                elections,
                locations='county_fips', 
                geojson=county_polygons,
                scope='usa', 
                color="r_votes", 
                animation_frame=elections["year"].astype(str), 
                color_continuous_scale=px.colors.diverging.balance, 
                range_color=[20, 80], 
                height=1000) 

Explore the map above. What do you notice about republican vote share, particularly as it relates to the previous map of unemployment?

Now we've got two datasets-- one on unemployment and another on election results. We want to merge them but CAREFUL: each row corresponds to the value of a variable $x$ in county $i$ and time $t$ (so, $x_{it}$); for example, the value in the first row of our dataset under the unemployment column would be $unemployment_{01001, 2000}$; i.e., the unemployment rate in Atauga County, Alabama (FIPS code 01001), in the year 2000. When our data has this structure ($x_{it}$), we call it **panel data**. It must be handled differently from **cross sectional data** ($x_i$), from merging to estimation. 

We can't just merge on $i$ or $t$, we need to merge on both. We can do so as follows: 

In [ ]:
df_c=pd.merge(elections,counties, on=['county_fips','year'])
df_c.head()

### Exercise

OK. Our data is clean and ready for analysis. Because we're going to be investigating the relationship between unempoyment rates and republican voteshare via a regression model, we're going to need to follow the four steps of regression modeling from [last week](https://oballinger.github.io/QM2/notebooks/W09.%20Linear%20Regression.html). 

First, formulate a research question (complete with null and alternative hypothesis), and then follow these steps for our dataset, `df_c` (bonus points if you account for the influence of population).

1. Summary Statistics
    * Table of Summary Statistics
2. Visualisation 
    * Exploratory Plots
3. Assumptions
    * A. Independence
    * B. Heteroscedasticity: Regression plots + Q-Q plot
    * C. Multicollinearity: VIF + Correlation Matrix 
4. Regression
    * Regression Table

For the moment, when you run the regression, ignore the fact that we have panel data and just run a regular regression of the form 
$$\huge Y= \beta_0 + \beta_1X+\epsilon $$

### Accounting for Space and Time

If you've done things correctly, you'll notice two things. First, the appears to be a generally negative relationship between unemployment and republican voteshare; in other words, places with higher unemployment tend to vote *against* republicans. Second, we've egregiously violated the independence assumption. We have repeat observations of the same individuals (counties) over time. As such, this result may be biased unless we account for space and time.

As we saw in the lecture, panel data actually contains *two* sources of variation: differences *between* individuals (in this case, counties), and *within* individuials. So, a simple research question such as "Does unemployment increase republican voteshare" is actually two different questions:

1. Does a higher level of unemployment lead to higher republican vote shares **between counties**?
2. Does an *increase* in the unemployment rate over time lead to an *increase* in republican vote shares **within counties**?

Neither is more important than the other, but we must be careful not to conflate them as they are very different questions. A straighforward way of answering the first question would be to get rid of the time dimension in our data by running a separate regression for each year: 

In [ ]:
models=[] # create empty list to store the models
names=[] # create empty list to store the names of the models

for year in range(2000,2021, 4): # loop through years from 2000 to 2020 in increments of 4
    election=df_c[df_c['year']==year] # subset the data to only include the year of interest
    model= ols('r_votes ~ unemployment + population', data=election).fit() # run a regression of the republican vote share on the unemployment rate
    models.append(model) # append the model to the list of models
    names.append(str(year)) # append the name of the model to the list of names

table=summary_col( # create a regression table 
    models, # pass the models to the summary_col function
    stars=True, # add stars denoting the p-values of the coefficient to the table; * p<0.05, ** p<0.01, *** p<0.001
    float_format='%0.3f', # set the decimal places to 3
    model_names=names, # set the names of the model
    info_dict = {"N":lambda x: "{0:d}".format(int(x.nobs))}) # add the number of observations to the table

print(table) # print the table

This table is pretty informative. Using what we learned from last week, we can say that for the 2020 election, 

* A 1% increase in the unemployment rate was associated with a 2.3% *decrease* in republican voteshare.
* A 1000-person increase in population was associated with 0.029% decrease in republican voteshare.
* both of these results are statistically signifiant at the 0.01 level. 
* 23% of the variation in republican voteshare can be explained by unemployment and population.

Crucially, "increase" in this context pertains to *differences in between counties*! 

We can also compare these results across different elections. The coefficient for the unemployment variable in 2020 is over twice the size of the same coefficient in 2016! So it looks like actually unemployment and republican voteshare are *negatively* related, contrary to popular belief. 

But is this the whole story? 

Below, i've isolated West Virginia, one of the states with the highest unemployment rates in America. Instead of drawing a new regression line every year, i've drawn a new regression line for each county over the six elections.         

In [ ]:
west_virginia=df_c[(df_c['state']==54)]
sns.lmplot(data=west_virginia, x='unemployment', y='r_votes', ci=None, hue='county', height=10, legend=False, palette='husl', scatter_kws={'alpha':.3})

Within a given county, an increase in the unemployment rate is associated with an **increase** in republican voteshare! This is where the second question comes in (variation within counties). 

We got away with doing a series of cross-sectional analyses (a new regression for each election) because we have over 3000 counties, so $n>3000$ for each of those regressions (though even so, we're still splitting our data up and it would be better to leverage the full dataset of >18000 observations in one regression). It also provides relatively useful information about the importance of unemployment across the country for each election. We can't really apply the same thinking to this situation, since we only have six time periods. If we ran a separate regression for each county, we would only have six observations per regression-- nowhere near enough to satisfy the central limit theorem (at least n>30). The insights would also be of limited utility; we would get over 3000 unique estimates for the realtionship between county-level employment and election results. Imagine trying to fit *that* into one table. 

Luckily, there's a way of modeling this relationship that allows us to account for differences in between counties, while also capturing the variation within counties. This is called a **Fixed Effect regression**

> **Fixed Effects Models**: In experimental research, unmeasured differences between subjects are often controlled for via random assignment to treatment and control groups. Hence, even if a variable like Socio-Economic Status is not explicitly measured, because of random assignment, we can be reasonably confident that the effects of SES are approximately equal for all groups. Of course, random assignment is usually not possible with most survey research. If we want to control for the effect of a variable, we must explicitly measure it. If we don’t measure it, we can’t control for it. In practice, there will almost certainly be some variables we have failed to measure (or have measured poorly), so our models will likely suffer from some degree of omitted variable bias. 
>When we have panel data (the same people/states/counties. etc. measured at two or more points in time) another alternative presents itself: we can use the subjects as their own controls. With panel data we can control for stable characteristics (i.e. characteristics that do not change across time) whether they are measured or not. These include such things as sex, race, and ethnicity for individuals, or urban/rural, topography, economic structure for geographic areas. The idea is that, whatever effect these variables have at one point in time, they will have the same effect at a different point in time because the values of such variables do not change. 

A fixed effect regression takes the following form: 

$$\huge Y_{it}=\alpha_i+\beta X_{it}+\epsilon_{it}$$

Where:
* $X_{it}$ are the independent variables (e.g. population and unemployment) whose values vary over time. 
* $\beta$ is the slope coefficient for variable $x$ (e.g. unemployment). The model assumes that these effects are time-invariant, e.g. the effect of $x$ is the same at same 1 as it is at time 4 (although the value of $x$ can be different at different time periods).
* $\alpha_i$ and $\epsilon_{it}$ are both error terms. $\epsilon_{it}$ is different for each individual at each point in time. $\alpha_i$ only varies across individuals but not across time. We can think of $\alpha_i$ as representing the effects of all the time invariant/stable variables that have NOT been included in the model. So, given that we have 6 time periods for each county then the six records for county 1 would all have the same value for $\alpha_1$, the six records for county 2 would all have the same value for $\alpha_2$, etc. But, $\epsilon_{it}$ is free to be different for every case at every time period.

A fixed effect regression allows us to account for $\alpha_i$ through a technique called **demeaning**

>**Demeaning**: After demeaning, all variables for all cases have a mean of 0. That means that all the between-subject variability has been eliminated. All that is left is the within-subject variability. So, with a fixed effects model, we are analyzing what causes individual’s values to change across time. Variables whose values do not change (like race or gender) cannot cause changes across time (unless their effects change across time as well). However, whatever effect they have at one time is the same effect that they have at other times, so the effects of such stable characteristics are controlled. 

In essence, you can picture this as allowing you to draw a separate regression line through each set of observations from the same group in your data (in this case, one county over time); however, while the *intercept* of these lines can vary (their absolute position), they will all have the same *slope* and will therefore be parallel. This is important, as we want to find one slope-- one common effect of x-- that fits *all* groups. 

Run the command below to install the library.

In [ ]:
!pip install linearmodels
!pip install typing-extensions==3.7.4.3

In [ ]:
from linearmodels import PanelOLS
from linearmodels import RandomEffects
import statsmodels.formula.api as smf
from linearmodels.panel import compare

df_c=df_c.set_index(['county_fips','year']) # set the index to the county fips code and the year
panel = PanelOLS.from_formula('r_votes ~ 1  + population + unemployment  + EntityEffects',df_c).fit() # run a fixed effects model
print(compare({'Fixed Effects': panel,}, stars=True)) # print the model formatted as a regression table

When accounting for time-invariant differences between counties, the effect of population remains negative. This suggests that counties in which the population is *decreasing* tend to experience an increase in republican voteshare. More specifically, for every 1000 people that leave a county, republican voteshare increases by 0.06%. 

The really interesting part of this regression table, however, is the coefficient on the unemployment variable, which is now positive. This suggests that-- once we account for the differences between counties-- an increase in the unemployment rate *within* a county is *positively* associated with republican voteshare. Indeed, a 1% increase in the unemployment rate leads to a 0.28% increase in republican voteshare. 

This regression output even gives us three separate $R^2$ values-- one for between-variation, another for within, and one overall. 